In [1]:
import os
#Set current location to the location of the script
os.chdir(os.path.dirname("/dust3r/playground.ipynb"))
print(os.getcwd())

/dust3r


In [2]:
#Display imgs
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image

In [3]:
from dust3r.inference import inference
from dust3r.model import AsymmetricCroCo3DStereo
from dust3r.utils.image import load_images
from dust3r.image_pairs import make_pairs
from dust3r.cloud_opt import global_aligner, GlobalAlignerMode

Warning, cannot find cuda-compiled version of RoPE2D, using a slow pytorch version instead


/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [15]:
device = 'cuda'
batch_size = 1
schedule = 'cosine'
lr = 0.01
niter = 300

model_name = "checkpoints/DUSt3R_ViTLarge_BaseDecoder_512_dpt.pth"
# you can put the path to a local checkpoint in model_name if needed
model = AsymmetricCroCo3DStereo.from_pretrained(model_name).to(device)
# load_images can take a list of images or a directory
images = load_images(['croco/assets/Chateau1.png', 'croco/assets/Chateau2.png'], size=512, verbose=True)

... loading model from checkpoints/DUSt3R_ViTLarge_BaseDecoder_512_dpt.pth
instantiating : AsymmetricCroCo3DStereo(enc_depth=24, dec_depth=12, enc_embed_dim=1024, dec_embed_dim=768, enc_num_heads=16, dec_num_heads=12, pos_embed='RoPE100', patch_embed_cls='PatchEmbedDust3R', img_size=(512, 512), head_type='dpt', output_mode='pts3d', depth_mode=('exp', -inf, inf), conf_mode=('exp', 1, inf), landscape_only=False)
<All keys matched successfully>
>> Loading a list of 2 images
 - adding croco/assets/Chateau1.png with resolution 224x224 --> 512x384
 - adding croco/assets/Chateau2.png with resolution 224x224 --> 512x384
 (Found 2 images)


In [5]:
pairs = make_pairs(images, scene_graph='complete', prefilter=None, symmetrize=True)
output = inference(pairs, model, device, batch_size=batch_size)

>> Inference with model on 2 image pairs


100%|██████████| 2/2 [00:02<00:00,  1.44s/it]


In [6]:
view1, pred1 = output['view1'], output['pred1']
view2, pred2 = output['view2'], output['pred2']

In [7]:
scene = global_aligner(output, device=device, mode=GlobalAlignerMode.PointCloudOptimizer)
loss = scene.compute_global_alignment(init="mst", niter=niter, schedule=schedule, lr=lr)

 init edge (1*,0*) score=48.34415817260742
 init loss = 0.006030057556927204
Global alignement - optimizing for:
['pw_poses', 'im_depthmaps', 'im_poses', 'im_focals']


100%|██████████| 300/300 [00:17<00:00, 17.08it/s, lr=1.27413e-06 loss=0.00275613]


In [8]:
imgs = scene.imgs
focals = scene.get_focals()
poses = scene.get_im_poses()
pts3d = scene.get_pts3d()
confidence_masks = scene.get_masks()

In [14]:
print(view1["img"][1])

tensor([[[-0.5608, -0.5765, -0.6000,  ...,  0.5451,  0.5843,  0.6000],
         [-0.6078, -0.6000, -0.5765,  ...,  0.6549,  0.6941,  0.7020],
         [-0.6941, -0.6863, -0.6627,  ...,  0.7490,  0.7490,  0.7490],
         ...,
         [-0.0510, -0.0510, -0.0510,  ..., -0.6235, -0.6471, -0.6549],
         [-0.0510, -0.0510, -0.0275,  ..., -0.6471, -0.6706, -0.6706],
         [-0.0275, -0.0353, -0.0275,  ..., -0.6549, -0.6627, -0.6627]],

        [[-0.4824, -0.4980, -0.5137,  ...,  0.6784,  0.7176,  0.7333],
         [-0.5294, -0.5216, -0.4902,  ...,  0.7882,  0.8196,  0.8353],
         [-0.6078, -0.6000, -0.5765,  ...,  0.8745,  0.8745,  0.8745],
         ...,
         [ 0.2000,  0.2000,  0.2000,  ..., -0.5294, -0.5451, -0.5529],
         [ 0.1843,  0.1922,  0.2078,  ..., -0.5529, -0.5686, -0.5686],
         [ 0.2000,  0.2000,  0.2078,  ..., -0.5451, -0.5608, -0.5608]],

        [[-0.7647, -0.7725, -0.7804,  ...,  0.8196,  0.8667,  0.8824],
         [-0.8039, -0.7961, -0.7647,  ...,  0

In [9]:
#Rescale images
img1 = view1["img"].numpy()
img2 = view2["img"].numpy().squeeze()
print(img1.shape)
img1 = Image.fromarray(np.uint8(img1 * 255))
img2 = Image.fromarray(np.uint8(img2 * 255))

plt.figure(figsize=(10, 10))
plt.subplot(1, 2, 1)
plt.imshow(img1)
plt.axis('off')

plt.subplot(1, 2, 2)
plt.imshow(img2)
plt.axis('off')

(2, 3, 384, 512)


TypeError: Cannot handle this data type: (1, 1, 384, 512), |u1